# Crawl DC Study Help for Verse Level from scripture central

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from urllib.parse import urljoin, urlparse

from bs4 import BeautifulSoup

from models.crawl_utils import get_page, save_page

In [ ]:
# config
host = 'https://doctrineandcovenantscentral.org/sections/'
base_dir = '../data/raw/dc_verse_level'
bs_parser = 'html.parser'
delay_seconds = 5
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

In [ ]:
def extract_hrefs_from_elementor_heading_title(html):
    hrefs = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all div tags with the class 'views-field-title'
    divs_with_class = soup.find_all('div', class_='elementor-heading-title')

    for div in divs_with_class:
        # Find all anchor tags within the div
        anchor_tags = div.find_all('a')

        # Extract href attribute from each anchor tag and append to the list
        for anchor in anchor_tags:
            href = anchor.get('href')
            if href:
                hrefs.append(urljoin(host, href))

    return hrefs

In [ ]:
def get_path(url):
    if not url.endswith('/'):
        url += '/'
    path_components = urlparse(url).path.split('/')
    return os.path.join(base_dir, f"{path_components[-2]}.json")
    

In [ ]:
study_links = []

status_code, html = get_page("https://doctrineandcovenantscentral.org/sections/", delay_seconds)
if status_code != 200:
    print("Error!", status_code)
hrefs = extract_hrefs_from_elementor_heading_title(html)
study_links.extend(hrefs)
print(len(study_links))
study_links[:5]
   

In [ ]:
def extract_hrefs(html, base_url):
    soup = BeautifulSoup(html, 'html.parser')
    
    target_text = "Verse-Level Helps"
    for div in soup.find_all('div'):
        if div.get_text(strip=True).startswith(target_text):
            next_sibling = div.find_next_sibling()
            if next_sibling and next_sibling.find('a', href=True):
                first_href = next_sibling.find('a', href=True)['href']
                first_href = first_href.split('?')[0]
                return urljoin(base_url, first_href)
    return None


In [ ]:
verse_links = []
for link in study_links:

    status_code, html_content = get_page(link, delay_seconds)
    if status_code != 200:
        print("Error!", status_code)
    href_value = extract_hrefs(html_content, link)
    print(href_value)
    verse_links.append(href_value)
print(len(verse_links))

In [ ]:
for url in verse_links:
    path_file =  get_path(url)
    print(path_file)
    if os.path.exists(path_file):
        continue
    status_code, html = get_page(url, delay_seconds)
    if status_code != 200:
        print("Error!", status_code , url)
        continue
    save_page(path_file,url,html)
    
print("End")